In [1]:
from IPython.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [2]:
import numpy as np
from desdeo_emo.EAs import RVEA, NSGAIII
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem import DataProblem
from desdeo_tools.utilities import fast_non_dominated_sort, hypervolume_indicator

import matplotlib.pyplot as plt
import sklearn
from pyDOE import lhs
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, WhiteKernel, RationalQuadratic, DotProduct, ConstantKernel, Matern

## Assignment 4

## Task 1 

Implement probability of selection for single objective optimization. * (refer eqn.
12, 13, 14)

In [3]:
np.set_printoptions(precision=4,suppress=True) # easier to check values rn when we dont care too much about the exactness

In [4]:
# in minimization think probability of being wrong, in max probability of acceptance
# probability of sample A dominating sample B in max is P(A > B) is
def prob_selection(a, b, sigma_a, sigma_b):
    #sigma_b = sigma_b + 0.01 # just to prevent division by zero
    # seems not to be the correct way of hanlding sigma 0
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b
    
    top = -2.5*m / np.sqrt(2 + 2*s**2)
    bot = 1 + np.exp(top)
    return 1 / bot

# b clearly dominates
a = np.array([1.0, 1.0])
b = np.array([0.5, 1.5])
sigma_a = .01
sigma_b = .01

prob_selection(a, b, sigma_a, sigma_b)

array([1., 0.])

In [5]:
def prob_sel2(a, b, sigma_a, sigma_b):
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b
    
    bot = 0.8*np.sqrt(2 + 2*s**2)
    return 0.5*(1 + np.tanh(m/bot))
    
    
    
# a1 clearly dominates b1, b1 clearly dominates a2
a = np.array([1.0, 1.0])
b = np.array([0.5, 1.5])
sigma_a = 0.001
sigma_b = 0.001

prob_sel2(a, b, sigma_a, sigma_b)

array([1., 0.])

So we can see that probability of being wrong with first items (1 and 2) by choosing a is 0.22. Again for second items (1,5) thats 0.007.  And also with 3rd items (3,1) its 0.92. So we can clearly see its working properly.

## Task 2

Implement probability of dominance and ranking. * (refer eqn. 15, 17, 18, 22)

- Generate 50 samples from DTLZ2 problem (3 objectives, 5 decision variables) using LHS sampling
- Build Kriging surrogates using the generated samples
- Resample or generate 20 data points randomly
- Rank the 20 points using probability of dominance ranking

In [9]:
dtlz2_3 = test_problem_builder("DTLZ2", n_of_variables=5, n_of_objectives=3)

In [10]:
def create_samples(dec_dim, samples):
    # create samples 
    x = lhs(dec_dim, samples)     
    return np.asarray(x)

In [11]:
x = create_samples(5, 50)
print(x.shape)
eval_results = dtlz2_3.evaluate(x)
y = eval_results.objectives
print(y[:5])

(50, 5)
[[0.8913 0.6913 0.0039]
 [0.1408 0.6218 1.0171]
 [0.8357 0.3682 0.6289]
 [0.0402 0.1886 1.1342]
 [0.1014 0.0899 1.2099]]


In [12]:
print(y[:,0])

[0.8913 0.1408 0.8357 0.0402 0.1014 0.5911 0.4447 0.86   1.1987 0.089
 0.3723 1.1387 0.7821 1.0289 0.0566 0.801  0.3459 0.758  0.1752 0.003
 0.2816 0.4041 0.1024 0.2507 0.6294 0.4027 0.8377 0.3641 0.0908 0.3179
 0.5446 0.4121 0.8814 0.0018 0.4028 0.7037 1.0465 1.3726 1.2104 0.4077
 0.3035 0.0579 0.2424 0.5002 0.0054 0.4853 0.3232 0.8696 0.5255 0.3682]


In [13]:
## build surrogates

kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 20.0), nu=1.5)
    
y1 = y[:,0]
gpr1 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y1)

y2 = y[:,1]
gpr2 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y2)

y3 = y[:,2]
gpr3 = GaussianProcessRegressor(kernel,n_restarts_optimizer=1,random_state=7).fit(x, y3)

In [14]:
## resampling

x_samples = create_samples(5, 20)

# predict
y1_sample, y1_std = gpr1.predict(x_samples, return_std=True) 

# predict
y2_sample, y2_std = gpr2.predict(x_samples, return_std=True) 

# predict
y3_sample, y3_std = gpr3.predict(x_samples, return_std=True) 

In [15]:


# eq 15: MO P(A < B) : product to k, P(Aj < Bj)
def smaller(a, b):
    p = 0
    # probl sel calculates P(A>B), so turn around for P (A < B)
    props = prob_selection(a, b, .01, .01) # should be prob a, b
    p = np.prod(props)
    #p = props
    #print(props)

    return p

# eq 17: MO P(A > B) : product to k, 1 - P(Aj < Bj)
def bigger(a, b):
    return 1 - smaller(a, b)


# eq 18: MO P(A = B) : 1 - P(A < B) - P(A > B)
def equals(a, b):
    return 1 - smaller(a, b) - bigger(a, b)


def me_equals(a, b):
    summa = 0
    
    #if f1 < f2:
    
def dominates(x: np.ndarray, y: np.ndarray):
    dom = 0

    for i in range(len(x)):
        if x[i] > y[i]:
            return 0
        elif x[i] < y[i]:
            dom = 1
            
    return dom

In [16]:
dominates(a,b)

0

In [20]:
a = np.array([0.35,0.5])
b = np.array([0.35,0.5])


B = a
E = b

smaller(B,E)

0.25

In [21]:
bigger(B,E)

0.75

In [22]:
bigger(a,b)

0.75

In [23]:
equals(a,b)

0.0

In [24]:
f = bigger(a, b)

# objective values from fitness measurements
f1 = np.array([0,1])
f2 = np.array([1,2])
# f1 dominates f2

In [25]:
rrr = np.array([1.,2.,3.])
kkk = np.array([4.,5.,6.])



In [26]:
def sum_of_rank_pos(n):
    return n*(n - 1)/2

sum_of_rank_pos(ff.shape[0])

15.0

In [27]:

# eq 21: give fitnesses, return rankings 
def eq21(f):
    ri = np.zeros_like(f)
    
    # FOR i in each individual. Sum compared to others j individuals -0.5 when i = j. 
    for i in range(f.shape[0]):
        for j in range(f.shape[0]):
            ri[i] += smaller(f[j], f[i])
            print(ri[i])
            #ri[i] += dominates(f[j], f[i])
        ri[i] -= 0.5
    return ri
    

# this is wrong or the equals atleast is wrong
def eq22(sols):
    print(sols.shape)
    num_sols = len(sols)
    ri = np.zeros(6)
    # could vectorize do stupid first
    print(ri.shape)
    #n = f.shape[0]
    for i in range(num_sols):
        #print(sols[i])
        for j in range(num_sols):
            ri[i] += dominates(sols[i], sols[j])    
            print(f"ri {ri[i]} __ {i} ___ {j}")
    
        ri[i] -= 0.5
    ri *= 0.5
            #ri[i] -= 0.5
        
            
    print(ri)
    #ri -= 0.5
    return ri
    
    

# eq 22: proba ranking : sum to n, P(Fj > Fi) + 0.5 sum to n P(Fj = Fi) - 0.5
def probabilitistic_ranking():
    ranks = 0
    #non_dom = 
    
    return ranks

In [25]:
A = [0.2, 0.9]  # tässä on a1 a2. kun lasketaan prod P (Ai < bi) i to n, tulee P(A < B)
B = [0.3, 0.5]
C = [0.5, 0.3]
D = [0.8, 0.2]
E = [0.85, 0.5]
F = [0.9, 0.95]

sols = np.array([A, B, C, D, E, F])
print(sols.shape)
ff = np.zeros(sols.shape[0])
# this being eq21
for i in range(ff.shape[0]):
    for j in range(ff.shape[0]):
        ff[i] += smaller(sols[j], sols[i])
    #ff[i] -= 0.5

(6, 2)


NameError: name 'smaller' is not defined

In [58]:
def prob_sel2(a, b, sigma_a, sigma_b):
    if sigma_b == 0.0:
        s = 0.
        m = np.zeros_like(a)
        print("hh")
    else:
        s = sigma_a / sigma_b
        m = (a - b) / sigma_b
    
    bot = 0.8*np.sqrt(2 + 2*s**2)
    res = 0.5*(1 + np.tanh(m/bot))
    res = np.prod(res)
    #print(res.shape)
    #print(res)
    return res


# eq 15: MO P(A < B) : product to k, P(Aj < Bj)
def P_wrong(a, b):
    p = 0
    # probl sel calculates P(A>B), so turn around for P (A < B)
    props = prob_sel2(a, b, .01, .01) # should be prob a, 
    
    #p = np.prod(props)
    p = props
    print(props)

    return p

# eq 17: MO P(A > B) : product to k, 1 - P(Aj < Bj)
def bigger(a, b):
    return 1 - smaller(a, b)


# eq 18: MO P(A = B) : 1 - P(A < B) - P(A > B)
def equals(a, b):
    return 1 - smaller(a, b) - bigger(a, b)


## DONE:

Single objective ranking works with uncertainty 0 and uncertainty 1. So prob_sel2 works
MO ranking with uncertainty 0 works

## TODO:

Multiobjective ranking with uncertainty has a bug and not working.

In [52]:
sigma_a, sigma_b = 1.,1.

def sof_prob_dom_single(fit, j):
    summa = 0
    for i in range(len(fit)):
        if j == fit[i]:
            #summa += 0.5
            summa += prob_sel2(j, fit[i], sigma_a, sigma_b)
        elif j > fit[i]:
            #summa += 1.
            summa += prob_sel2(j, fit[i], sigma_a, sigma_b)
        elif j < fit[i]:
            #summa += 0
            summa += prob_sel2(j, fit[i], sigma_a, sigma_b)
        else:
            print("should not get here")
    summa -= 0.5
    return summa
  

In [39]:
# test from beginnning
fit_values=[1.,2., 3., 3., 5., 6., 6.5 ]
s = []

for i in range(len(fit_values)):
    s.append(sof_prob_dom_single(fit_values, fit_values[i]))
s

()
0.5
()
0.22270013882530887
()
0.07585818002124356
()
0.07585818002124356
()
0.006692850924284843
()
0.0019267346633274895
()
0.001032231036754816
()
0.7772998611746911
()
0.5
()
0.22270013882530887
()
0.22270013882530887
()
0.022977369910025636
()
0.006692850924284843
()
0.003593602581420108
()
0.9241418199787564
()
0.7772998611746911
()
0.5
()
0.5
()
0.07585818002124356
()
0.022977369910025636
()
0.012431650853185816
()
0.9241418199787564
()
0.7772998611746911
()
0.5
()
0.5
()
0.07585818002124356
()
0.022977369910025636
()
0.012431650853185816
()
0.9933071490757152
()
0.9770226300899744
()
0.9241418199787564
()
0.9241418199787564
()
0.5
()
0.22270013882530887
()
0.1329642401978292
()
0.9980732653366725
()
0.9933071490757152
()
0.9770226300899744
()
0.9770226300899744
()
0.7772998611746911
()
0.5
()
0.34864513533394575
()
0.9989677689632452
()
0.99640639741858
()
0.9875683491468141
()
0.9875683491468141
()
0.8670357598021707
()
0.6513548646660543
()
0.5


[0.38406831549216314,
 1.2559639622410395,
 2.3127088819379025,
 2.3127088819379025,
 4.174277798146341,
 5.071370671100974,
 5.488901489143679]

In [53]:
def dominates(x: np.ndarray, y: np.ndarray):
    dom = 0

    for i in range(len(x)):
        if x[i] > y[i]:
            return -1
        elif x[i] < y[i]:
            dom = 1
            
    return dom


x = np.array([1., 1.])
y = np.array([.1, 1.])
dominates(x, y)

-1

In [201]:
sigma_a, sigma_b = 1.,1.


# feels like im missing the product
def sof_prob_dom_multi2(fit, j):
    summa = 0
    # should just go over the multi
    for i in range(len(fit)):
        for ii in range(fit.shape[1]):
            #print("i", ii)
            #print("j",j)
            #j = j[ii]
            fits = fit[i][ii]
            if j[ii] == fits:
                summa += 0.25
                #s = prob_sel2(j[ii], fits, sigma_a, sigma_b)
                #summa += s
            elif j[ii] > fits:
                summa += 0.5
                #summa += prob_sel2(j[ii], fits, sigma_a, sigma_b)
            elif j[ii] < fits:
                summa += 0
                #summa += prob_sel2(j[ii], fits, sigma_a, sigma_b)
            else:
                print("should not get here")
        
        #summa *= 0.5
    summa -= 0.5
    return summa


# feels like im missing the product
def sof_prob_dom_multi(fit, j):
    summa = 0
    # should just go over the multi
    for i in range(len(fit)):

        #print("j",j)
        # P( A == B)
        #print("round")
        if dominates(j, fit[i]) == 0:
            xx = prob_sel2(j, fit[i], sigma_a, sigma_b)
            #print(xx)
            summa += 1.
        # P( A > B)  
        elif dominates(j, fit[i]) == 1:
            xx2 = prob_sel2(j, fit[i], sigma_a, sigma_b)
            #print(xx2)
            summa += prob_sel2(j, fit[i], sigma_a, sigma_b)
        # P( A < B)
        elif dominates(j, fit[i]) == -1:
            xx3 = prob_sel2(j, fit[i], sigma_a, sigma_b)
            #print(xx3)
            summa += prob_sel2(j, fit[i], sigma_a, sigma_b)
        else:
            print("should not get here")
         

    summa *= 0.5
    #summa -= 0.5
    return summa


###  this
If the tness values are non-dominated, the corresponding
probabilities are P (A > B ) = 0, P (A < B ) = 0, and P (A  B ) = 1. We have
now lost the value 1 from the probability of domination calculations, therefore

In [202]:

s = []

for i in range(len(sols)):
    s.append(sof_prob_dom_multi(sols, sols[i]))
s

[1.064357939029948,
 0.9685302625412685,
 0.9702339537728406,
 1.0361020890956514,
 1.2086127540686513,
 1.468952935018257]

In [203]:
s2 = []

for i in range(len(sols)):
    s2.append(sof_prob_dom_multi2(sols, sols[i]))
s2

[2.0, 1.75, 1.5, 1.5, 3.25, 5.0]

## ^^ above works for uncertainty 0

In [204]:
s = np.asarray(s)
s2 = np.asarray(s2)
s + s2

array([3.0644, 2.7185, 2.4702, 2.5361, 4.4586, 6.469 ])

In [124]:
## ^^ results look correct if eq22 would be correct...

In [30]:
ff
#pr1 = ff

array([0., 0., 0., 0., 0., 0.])

In [30]:
pr1 = eq21(ff)
pr1

0.5
1.0001164570415275
1.5002329140830533
2.0002329140830533
2.0002329140830533
2.0002329140830533
0.4998835429584724
0.9998835429584725
1.4998835429584707
1.9997670859169432
1.9997670859169432
1.9997670859169432
0.4998835429584742
0.999883542958476
1.499883542958476
1.9997670859169503
1.9997670859169503
1.9997670859169503
0.5
1.0001164570415275
1.5002329140830533
2.0002329140830533
2.0002329140830533
2.0002329140830533
1.0
2.0
3.0
4.0
4.5
4.5
1.0
2.0
3.0
4.0
5.0
5.5


array([1.5002, 1.4998, 1.4998, 1.5002, 4.    , 5.    ])

In [31]:
pr2 = eq22(sols)
pr2

(6, 2)
(6,)
ri 0.0 __ 0 ___ 0
ri 0.0 __ 0 ___ 1
ri 0.0 __ 0 ___ 2
ri 0.0 __ 0 ___ 3
ri 0.0 __ 0 ___ 4
ri 1.0 __ 0 ___ 5
ri 0.0 __ 1 ___ 0
ri 0.0 __ 1 ___ 1
ri 0.0 __ 1 ___ 2
ri 0.0 __ 1 ___ 3
ri 1.0 __ 1 ___ 4
ri 2.0 __ 1 ___ 5
ri 0.0 __ 2 ___ 0
ri 0.0 __ 2 ___ 1
ri 0.0 __ 2 ___ 2
ri 0.0 __ 2 ___ 3
ri 1.0 __ 2 ___ 4
ri 2.0 __ 2 ___ 5
ri 0.0 __ 3 ___ 0
ri 0.0 __ 3 ___ 1
ri 0.0 __ 3 ___ 2
ri 0.0 __ 3 ___ 3
ri 1.0 __ 3 ___ 4
ri 2.0 __ 3 ___ 5
ri 0.0 __ 4 ___ 0
ri 0.0 __ 4 ___ 1
ri 0.0 __ 4 ___ 2
ri 0.0 __ 4 ___ 3
ri 0.0 __ 4 ___ 4
ri 1.0 __ 4 ___ 5
ri 0.0 __ 5 ___ 0
ri 0.0 __ 5 ___ 1
ri 0.0 __ 5 ___ 2
ri 0.0 __ 5 ___ 3
ri 0.0 __ 5 ___ 4
ri 0.0 __ 5 ___ 5
[ 0.25  0.75  0.75  0.75  0.25 -0.25]


array([ 0.25,  0.75,  0.75,  0.75,  0.25, -0.25])

In [32]:
res = pr1 + pr2
res

array([1.7502, 2.2498, 2.2498, 2.2502, 4.25  , 4.75  ])

In [33]:
np.sum(res)

17.5

In [432]:
assert np.floor(np.sum(res)) == sum_of_rank_pos(ff.shape[0])